In [3]:
# import os
# os.chdir('/')
# if not os.path.exists("/clifford-group-equivariant-neural-networks"):
#     !git clone https://github.com/DavidRuhe/clifford-group-equivariant-neural-networks.git
# os.chdir("/clifford-group-equivariant-neural-networks")

import sys
sys.path.append('clifford-group-equivariant-neural-networks')

In [4]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import torch
from algebra.cliffordalgebra import CliffordAlgebra
from models.modules.linear import MVLinear
from models.modules.gp import SteerableGeometricProductLayer
from models.modules.mvlayernorm import MVLayerNorm
from models.modules.mvsilu import MVSiLU
from models.nbody_cggnn import CEMLP
import torch
import torch.nn as nn
import torch.optim as optim
import math

In [5]:
# Define the metric for 3D space (Euclidean)
metric = [1, 1, 1]
d = len(metric)

# Initialize the Clifford Algebra for 3D
clifford_algebra = CliffordAlgebra(metric)

In [6]:
def make_edge_attr(node_features, edges, batch_size):
    edge_attributes = []

    total_number_edges = edges[0].shape[0]

    # Loop over all edges
    for i in range(total_number_edges):

        node1 = edges[0][i]
        node2 = edges[1][i]

        # difference between node features
        node_i_features = node_features[node1]  # [#features(charge, loc, vel), dim]
        node_j_features = node_features[node2]  # [#features(charge, loc, vel), dim]
        difference = node_i_features - node_j_features

        # geom product between node features
        geom_product = clifford_algebra.geometric_product(node_i_features, node_j_features)

        # now just add the 2 to get the "representation of the edge"
        # Stack all
        edge_representation = torch.cat((difference, geom_product), dim=0)
        edge_attributes.append(edge_representation)

    edge_attributes = torch.stack(edge_attributes)
    return edge_attributes


In [7]:
def embed_nbody_graphs(batch):
    loc, vel, edge_attr, charges, loc_end, edges = batch

    batch_size, n_nodes, _ = loc.size()

    # put the mean of the pointcloud on the origin (WHY?)
    loc_mean = loc - loc.mean(dim=1, keepdim=True)     # [batch, nodes, dim]

    # ALL THESE do [batch, nodes, dim] ->>> [batch * nodes, dim]
    loc_mean = loc_mean.float().view(-1, *loc_mean.shape[2:])
    loc = loc.float().view(-1, *loc.shape[2:])
    vel = vel.float().view(-1, *vel.shape[2:])
    edge_attr = edge_attr.float().view(-1, *edge_attr.shape[2:])
    charges = charges.float().view(-1, *charges.shape[2:])
    loc_end = loc_end.float().view(-1, *loc_end.shape[2:])

    invariants = charges
    invariants = clifford_algebra.embed(invariants, (0,))
    charges_in_clifford = invariants

    xv = torch.stack([loc_mean, vel], dim=1) # now of the shape [batch * nodes, 2 (because its loc mean as well as vel), dim]
    covariants = clifford_algebra.embed(xv, (1, 2, 3))
    pos_vel_in_clifford = covariants

    nodes_in_clifford = torch.cat([invariants[:, None], covariants], dim=1) # [batch * nodes, #features(charge, loc, vel), dim]

    batch_index = torch.arange(batch_size, device=loc_mean.device) # torch.arange(batch_size) generates a tensor from 0 to batch_size - 1, creating a sequence that represents each graph in the batch. If batch_size is 3, this tensor will be [0, 1, 2]
    edges = edges + n_nodes * batch_index[:, None, None] # creates separate edge number for every graph. so if edge for graph 1 is between 3 and 4, graph 2 will be between 8 and 9 (if n_nodes = 5)
    edges = tuple(edges.transpose(0, 1).flatten(1)) # where the first element of the tuple contains all start nodes and the second contains all end nodes for edges across the entire batch. ([edges*batch], [edges*batch])

    extra_edge_attr_clifford = make_edge_attr(nodes_in_clifford, edges, batch_size) # [batch*edges, #numfeatures, difference + geomprod, dim]

    orig_edge_attr_clifford = clifford_algebra.embed(edge_attr[..., None], (0,)) # now [batch * edges, 1, dim]

    return nodes_in_clifford, extra_edge_attr_clifford, orig_edge_attr_clifford, loc_end


In [8]:
def generate_mock_batch(batch_size):
    """
    Generate a mock batch of data with specified shapes.

    Parameters:
    - batch_size (int): The size of the batch to generate.

    Returns:
    - A tuple containing tensors for loc_frame_0, vel_frame_0, edge_attr, charges, loc_frame_T, and edges.
    """
    # Constants
    n_nodes = 5  # Number of nodes
    n_features = 3  # Number of spatial features (e.g., x, y, z)
    n_edges = 20  # Number of edges
    n_edge_features = 1  # Number of features per edge

    # Generate data
    loc_frame_0 = torch.rand(batch_size, n_nodes, n_features)
    vel_frame_0 = torch.rand(batch_size, n_nodes, n_features)
    edge_attr = torch.rand(batch_size, n_edges, n_edge_features)
    charges = torch.rand(batch_size, n_nodes, 1)
    loc_frame_T = torch.rand(batch_size, n_nodes, n_features)

    # Generate edges indices
    # For simplicity, assuming all batches share the same structure of graph
    rows = torch.randint(0, n_nodes, (n_edges,))
    cols = torch.randint(0, n_nodes, (n_edges,))
    edges = torch.stack([rows, cols], dim=0).repeat(batch_size, 1, 1)  # Repeat the edge structure across the batch

    return loc_frame_0, vel_frame_0, edge_attr, charges, loc_frame_T, edges

In [9]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, batch_size):
        super(PositionalEncoding, self).__init__()

        # Adding a binary feature
        # Nodes get a [1, 0] and edges get a [0, 1]
        # node_marker = torch.zeros(5*batch_size, 1, 8)
        edge_marker = torch.ones(20*batch_size, 1, 8)

        # Concatenate this new feature
        # self.pe = torch.cat((node_marker, edge_marker), dim=0)
        self.pe = edge_marker

    def forward(self, x):
        print(x.shape, self.pe.shape)
        return torch.cat((x, self.pe), dim=1)


In [10]:
def scaled_dot_product_attention(query, key, value, mask=None):
    d_k = query.size(-1)
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9)
    attn = torch.softmax(scores, dim=-1)
    output = torch.matmul(attn, value)
    return output, attn


In [11]:
class GAST_block(nn.Module):
    def __init__(self, d_model, num_heads, clifford_algebra, channels):
        super(GAST_block, self).__init__()
        self.mvlayernorm = MVLayerNorm(clifford_algebra, channels)
        self.self_attn = #TBA
        self.mvlayernorm2 = MVLayerNorm(clifford_algebra, channels)
        self.cemlp = CEMLP(clifford_algebra, in_features, hidden_features, out_features)



    def forward(self, src, src_mask=None):
        src = self.mvlayernorm(src)



        return src


class GAST(nn.Module):
    def __init__(self, num_layers, d_model, num_heads,  clifford_algebra, channels):
        super(GAST, self).__init__()
        self.layers = nn.ModuleList([GAST_block(d_model, num_heads) for _ in range(num_layers)])
        self.norm = nn.LayerNorm(d_model)

    def forward(self, src, src_mask=None):
        for layer in self.layers:
            src = layer(src, src_mask)
        return self.norm(src)

SyntaxError: invalid syntax (1937155272.py, line 5)

In [12]:
class NBODY_Transformer(nn.Module):
    def __init__(self, input_dim, d_model, num_heads, num_layers, mv_in_features, mv_out_features, clifford_algebra, channels):
        super(NBODY_Transformer, self).__init__()
        self.positional_encoding = PositionalEncoding(d_model, batch_size)
        self.GAST = GAST(num_layers, d_model, num_heads, clifford_algebra, channels)

        # subspaces is false
        self.embedding = MVLinear(clifford_algebra, mv_out_features, mv_out_features, subspaces=False)
        self.fc = MVLinear(clifford_algebra, d_model, input_dim, subspaces = True)

        # NOG TE FIXEN
        self.edge_to_node_shape = MVLinear(clifford_algebra, mv_in_features, mv_out_features, subspaces=True)

    def forward(self, nodes_in_clifford, edges_in_clifford, src_mask, batch_size):

        # POSITIONAL ENCODING NOG OVER TE SPREKEN
        edges_in_clifford = self.positional_encoding(edges_in_clifford)

        fully_embedded_edges = self.edge_to_node_shape(edges_in_clifford)
        src = torch.cat((nodes_in_clifford, fully_embedded_edges), dim=0)

        src = self.embedding(src)
        enc_output = self.GAST(src, src_mask)

        # LATER FIXEN
        output = self.fc(enc_output)


        return output


In [13]:
# Hyperparameters
input_dim = 8  # feature_dim
d_model = 512
num_heads = 8
num_layers = 6
mv_in_features = 8
mv_out_features = 3
channels = 8 # clifford algrebra vector


# Create the model
model = NBODY_Transformer(input_dim, d_model, num_heads, num_layers, mv_in_features, mv_out_features, clifford_algebra, channels)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

batch_size = 1
batch = generate_mock_batch(batch_size)
src_mask = None

# Training loop
model.train()
for epoch in range(10):
    optimizer.zero_grad()
    batch = batch
    components = embed_nbody_graphs(batch)
    nodes_in_clifford, extra_edge_attr_clifford, orig_edge_attr_clifford, loc_end = components
    edges_in_clifford = torch.cat((extra_edge_attr_clifford, orig_edge_attr_clifford), dim=1)
    tgt = loc_end

    output = model(nodes_in_clifford, edges_in_clifford, src_mask, batch_size)


    # OUTPUT WEER TERUG NAAR TFT SHAPE [5, 3] = [nodes*batches, 3]

    # originele locatie nog optellen bij prediction en dan dat vergelijken!!
    loss = criterion(output, tgt)
    loss.backward()
    optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {loss.item()}')


NameError: name 'batch_size' is not defined